In [55]:
import pandas as pd
from datetime import datetime as dt

def prep_data():
    df = pd.read_csv(filename, sep=separater, names=names)
    # covert time to datetime column
    df['time'] = pd.to_datetime(df['time'])
    # extract datetime to time window
    df['day'] = df.time.dt.dayofweek
    df['hour'] = df.time.dt.hour

    # filtering user&venue with less than 10 check-ins
    df = (df
     .merge(df.groupby('vid').uid.nunique().reset_index().rename(columns={'uid': 'num_uniq_uid'}), on='vid', how='left'))
    df = df[(df.num_uniq_uid >= 10) ]
    df = (df.merge(df.groupby('uid').vid.nunique().reset_index().rename(columns={'vid': 'num_uniq_vid'}), on='uid', how='left'))
    df = df[(df.num_uniq_vid >= 10) ]


    # indexing uid&vid and sort by uid and time
    dataset = df.merge(pd.Series(df.uid.unique()).reset_index().rename(columns={'index': 'new_uid', 0:'uid'}), left_on='uid', right_on='uid').merge(pd.Series(df.vid.unique()).reset_index().rename(columns={'index': 'new_vid', 0:'vid'}), left_on='vid', right_on='vid')
    del dataset['uid']
    del dataset['vid']
    dataset = dataset.rename(columns={'new_uid':'uid', 'new_vid':'vid'})
    uNum = dataset.uid.nunique()
    vNum = dataset.vid.nunique()
    sparsity = (dataset.shape[0] / (uNum*vNum))*100
    print("sparsity  %.2f% %" % sparsity)
    sort = dataset[['uid','vid','hour','day','time']].sort_values(['uid','time'])

    # generate userCheckins
    gb = sort.groupby('uid')    
    userCheckins = [gb.get_group(x) for x in gb.groups]


    for user in range(uNum):
        split_index = int(len(userCheckins[user])*ratio)
        train = userCheckins[user][:split_index]
        test = userCheckins[user][split_index:]
        train.to_csv(output+"Train", sep="\t", header=False, index=False, mode="a+")
        test.to_csv(output+"Test", sep="\t", header=False, index=False, mode="a+")


filename = '../../checkins/standford_checkins/loc-gowalla_totalCheckins.txt'
separater = '\t'
names = ['uid','time','lat','lon','vid']
ratio = 0.8
output = "gowalla"
prep_data()

sparsity  0.09%
